# Load images into table

This demonstrates different ways to load images into a database table.

We use the script called <em>madlib_image_loader.py</em> located at https://github.com/apache/madlib-site/tree/asf-site/community-artifacts/Deep-learning which uses the Python Imaging Library so supports multiple formats http://www.pythonware.com/products/pil/

## Table of contents

<a href="#setup">1. Setup image loader</a>

<a href="#fetch_numpy">2. Fetch images then load NumPy array into table</a>

<a href="#file_system">3. Load from file system into table</a>

In [1]:
%load_ext sql

/Users/fmcquillan/anaconda/lib/python2.7/site-packages/IPython/config.py:13: ShimWarning: The `IPython.config` package has been deprecated since IPython 4.0. You should import from traitlets.config instead.
  "You should import from traitlets.config instead.", ShimWarning)
/Users/fmcquillan/anaconda/lib/python2.7/site-packages/IPython/utils/traitlets.py:5: UserWarning: IPython.utils.traitlets has moved to a top-level traitlets package.
  warn("IPython.utils.traitlets has moved to a top-level traitlets package.")


In [2]:
# Greenplum Database 5.x on GCP for deep learning (PM demo machine)
#%sql postgresql://gpadmin@35.239.240.26:5432/madlib
        
# PostgreSQL local
%sql postgresql://fmcquillan@localhost:5432/madlib

u'Connected: fmcquillan@madlib'

In [3]:
%sql select madlib.version();
#%sql select version();

1 rows affected.


version
"MADlib version: 1.16, git revision: rc/1.16-rc1, cmake configuration time: Mon Jul 1 17:45:09 UTC 2019, build type: Release, build system: Darwin-16.7.0, C compiler: Clang, C++ compiler: Clang"


<a id="setup"></a>
# 1. Set up image loader

We use the script called <em>madlib_image_loader.py</em> located at https://github.com/apache/madlib-site/tree/asf-site/community-artifacts/Deep-learning

In [4]:
import sys
import os
from keras.datasets import cifar10

Using TensorFlow backend.


Couldn't import dot_parser, loading of dot files will not be possible.


In [5]:
madlib_site_dir = '/Users/fmcquillan/Documents/Product/MADlib/Demos/data'
sys.path.append(madlib_site_dir)

In [6]:
# Import image loader module
from madlib_image_loader import ImageLoader, DbCredentials

In [7]:
# Specify database credentials, for connecting to db
#db_creds = DbCredentials(user='gpadmin',
#                         host='35.239.240.26',
#                         port='5432',
#                         password='')

# Specify database credentials, for connecting to db
db_creds = DbCredentials(user='fmcquillan',
                          host='localhost',
                          port='5432',
                          password='')

In [8]:
# Initialize ImageLoader (increase num_workers to run faster)
iloader = ImageLoader(num_workers=5, db_creds=db_creds)

<a id="fetch_numpy"></a>
# 2. Fetch images then load NumPy array into table

<em>iloader.load_dataset_from_np(data_x, data_y, table_name, append=False)</em>

- <em>data_x</em> contains image data in np.array format


- <em>data_y</em> is a 1D np.array of the image categories (labels).


- If the user passes a <em>table_name</em> while creating ImageLoader object, it will be used for all further calls to load_dataset_from_np.  It can be changed by passing it as a parameter during the actual call to load_dataset_from_np, and if so future calls will load to that table name instead.  This avoids needing to pass the table_name again every time, but also allows it to be changed at any time.

           
- <em>append=False</em> attempts to create a new table, while <em>append=True</em> appends more images to an existing table.

In [12]:
# Load dataset into np array
(x_train, y_train), (x_test, y_test) = cifar10.load_data()

In [13]:
%sql DROP TABLE IF EXISTS cifar_10_train_data, cifar_10_test_data;

# Save images to temporary directories and load into database
iloader.load_dataset_from_np(x_train, y_train, 'cifar_10_train_data', append=False)
iloader.load_dataset_from_np(x_test, y_test, 'cifar_10_test_data', append=False)

Done.
MainProcess: Connected to madlib db.
Executing: CREATE TABLE cifar_10_train_data (id SERIAL, x REAL[], y TEXT)
CREATE TABLE
Created table cifar_10_train_data in madlib db
Spawning 5 workers...
Initializing PoolWorker-1 [pid 32578]
PoolWorker-1: Created temporary directory /tmp/madlib_7llkr1iwrB
Initializing PoolWorker-2 [pid 32579]
PoolWorker-2: Created temporary directory /tmp/madlib_gFBeKWnLLL
Initializing PoolWorker-3 [pid 32580]
PoolWorker-3: Created temporary directory /tmp/madlib_3EqOi1Elxy
PoolWorker-1: Connected to madlib db.
Initializing PoolWorker-4 [pid 32581]
PoolWorker-4: Created temporary directory /tmp/madlib_t3HzIGOeAn
PoolWorker-2: Connected to madlib db.
Initializing PoolWorker-5 [pid 32583]
PoolWorker-3: Connected to madlib db.
PoolWorker-5: Created temporary directory /tmp/madlib_I9wQ0HGhrx
PoolWorker-4: Connected to madlib db.
PoolWorker-5: Connected to madlib db.
PoolWorker-1: Wrote 1000 images to /tmp/madlib_7llkr1iwrB/cifar_10_train_data0000.tmp
PoolWorker

PoolWorker-3: Removed temporary directory /tmp/madlib_3EqOi1Elxy
PoolWorker-1: Removed temporary directory /tmp/madlib_7llkr1iwrB
PoolWorker-4: Removed temporary directory /tmp/madlib_t3HzIGOeAn
Done!  Loaded 50000 images in 24.7632198334s
5 workers terminated.
MainProcess: Connected to madlib db.
Executing: CREATE TABLE cifar_10_test_data (id SERIAL, x REAL[], y TEXT)
CREATE TABLE
Created table cifar_10_test_data in madlib db
Spawning 5 workers...
Initializing PoolWorker-6 [pid 32590]
PoolWorker-6: Created temporary directory /tmp/madlib_ukKJGqIRI7
Initializing PoolWorker-7 [pid 32591]
PoolWorker-7: Created temporary directory /tmp/madlib_FiXPZxXK47
Initializing PoolWorker-8 [pid 32592]
Initializing PoolWorker-9 [pid 32593]
PoolWorker-8: Created temporary directory /tmp/madlib_EVmSLqZ6Od
PoolWorker-6: Connected to madlib db.
PoolWorker-9: Created temporary directory /tmp/madlib_b4uua9iT9p
PoolWorker-7: Connected to madlib db.
Initializing PoolWorker-10 [pid 32595]
PoolWorker-8: Connec

<a id="file_system"></a>
# 3. Load from file system

Uses the Python Imaging Library so supports multiple formats
http://www.pythonware.com/products/pil/

<em>load_dataset_from_disk(root_dir, table_name, num_labels='all', append=False)</em>

- Calling this function  will look in <em>root_dir</em> on the local disk of wherever this is being run.  It will skip over any files in that directory, but will load images contained in each of its subdirectories.  The images should be organized by category/class, where the name of each subdirectory is the label for the images contained within it.


- The <em>table_name</em> and <em>append</em> parameters are the same as above  The parameter <em>num_labels</em> is an optional parameter which can be used to restrict the number of labels (image classes) loaded, even if more are found in <em>root_dir</em>.  For example, for a large dataset you may have hundreds of labels, but only wish to use a subset of that containing a few dozen.

For example, if we put the CIFAR-10 training data is in 10 subdirectories under directory <em>cifar10</em>, with one subdirectory for each class:

In [9]:
%sql drop table if exists cifar_10_train_data_filesystem;
# Load images from file system
iloader.load_dataset_from_disk('/Users/fmcquillan/tmp/cifar10', 'cifar_10_train_data_filesystem', num_labels='all', append=False)

Done.
MainProcess: Connected to madlib db.
Executing: CREATE TABLE cifar_10_train_data_filesystem (id SERIAL, x REAL[], y TEXT,                        img_name TEXT)
CREATE TABLE
Created table cifar_10_train_data_filesystem in madlib db
.DS_Store is not a directory, skipping
number of labels = 10
Found 10 image labels in /Users/fmcquillan/tmp/cifar10
Spawning 5 workers...
Initializing PoolWorker-1 [pid 64905]
PoolWorker-1: Created temporary directory /tmp/madlib_lFwfdAIfkq
Initializing PoolWorker-2 [pid 64906]
PoolWorker-2: Created temporary directory /tmp/madlib_PAZRj5SNyn
Initializing PoolWorker-3 [pid 64907]
PoolWorker-3: Created temporary directory /tmp/madlib_x0attqv2T6
PoolWorker-1: Connected to madlib db.
Initializing PoolWorker-4 [pid 64908]
PoolWorker-4: Created temporary directory /tmp/madlib_lyhVMiyvdL
Initializing PoolWorker-5 [pid 64909]
PoolWorker-5: Created temporary directory /tmp/madlib_8IpNResiVI
PoolWorker-3: Connected to madlib db.
PoolWorker-2: Connected to madlib 

PoolWorker-1: Loaded 1000 images into cifar_10_train_data_filesystem
PoolWorker-4: Loaded 1000 images into cifar_10_train_data_filesystem
PoolWorker-5: Loaded 1000 images into cifar_10_train_data_filesystem
PoolWorker-2: Loaded 1000 images into cifar_10_train_data_filesystem
PoolWorker-3: Loaded 1000 images into cifar_10_train_data_filesystem
PoolWorker-5: Wrote 1000 images to /tmp/madlib_8IpNResiVI/cifar_10_train_data_filesystem0009.tmp
PoolWorker-2: Wrote 1000 images to /tmp/madlib_PAZRj5SNyn/cifar_10_train_data_filesystem0009.tmp
PoolWorker-4: Wrote 1000 images to /tmp/madlib_lyhVMiyvdL/cifar_10_train_data_filesystem0009.tmp
PoolWorker-1: Wrote 1000 images to /tmp/madlib_lFwfdAIfkq/cifar_10_train_data_filesystem0009.tmp
PoolWorker-3: Wrote 1000 images to /tmp/madlib_x0attqv2T6/cifar_10_train_data_filesystem0009.tmp
PoolWorker-5: Loaded 1000 images into cifar_10_train_data_filesystem
PoolWorker-2: Loaded 1000 images into cifar_10_train_data_filesystem
PoolWorker-4: Loaded 1000 images

In [10]:
%%sql
SELECT COUNT(*) FROM cifar_10_train_data_filesystem;

1 rows affected.


count
50000
